In [1]:
from GPTmuseVAE import GPTmuseVAE

import os
import torch
from GPTmuseVAE import GPTmuseVAE
from miditok.pytorch_data import DatasetTok
from miditok import REMI
from torchtoolkit.data import create_subsets
from pathlib import Path
from utils import *
import pygame
from pygame import mixer

/Users/sebastianballesteros/Desktop/ie_classes/Semester_7/Advanced_AI/group_project_final/GPfinalvenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Hyperparameters
block_size = 124 # what is the maximum context length for predictions?
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# ------------

In [3]:
tokenizer = REMI(params= Path('midi_dataset_tokenizer_bpe.conf'))
vocab_size = len(tokenizer)

tokens_paths = list(Path('midi_dataset_tokens_no_bpe').glob("**/*.json"))

dataset = DatasetTok(
    tokens_paths, 
    max_seq_len=block_size, # to make target and prediction match the song length of block size
    min_seq_len=block_size, 
    one_token_stream= False,
    func_to_get_labels = get_artist_label
)

Loading data: midi_dataset_tokens_no_bpe/midi_metal/Slayer:   0%|          | 0/511 [00:00<?, ?it/s]

Loading data: midi_dataset_tokens_no_bpe/midi_metal/Slayer: 100%|██████████| 511/511 [00:02<00:00, 231.46it/s]


In [4]:
model = GPTmuseVAE( vocab_size= len(tokenizer),
                    n_embd = 32,
                    n_head = 4,
                    n_layer = 2,
                    z_dim = 32,
                    block_size = block_size,
                    dropout = 0.2)


m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')



0.166608 M parameters


In [5]:
loaded_state_dict = torch.load('checkpoints/checkpoint_3900.pt')
model.load_state_dict(loaded_state_dict['model_state_dict'])

<All keys matched successfully>

In [6]:
gen_seed = torch.stack(dataset[0:2]['input_ids'])
gen_seed

tensor([[  4, 137, 198,   4,   4, 153,  20, 108, 118,  22, 108, 118,  33, 108,
         118, 161,  20, 108, 118,  22, 108, 118,  33, 108, 118,   4,   4,   4,
         137,  29, 108, 118, 139,  29, 108, 118, 141,  29, 108, 118,   4, 153,
          20, 108, 118,  22, 108, 118,  33, 108, 118, 161,  20, 108, 118,  22,
         108, 118,  33, 108, 118,   4, 165,  22, 108, 118, 167,  22, 108, 118,
           4, 137,  20, 108, 118,  33, 108, 118, 139,  22, 108, 118, 141,  22,
         108, 118, 143,  22, 108, 118, 145,  29, 108, 118, 147,  29, 108, 118,
         149,  22, 108, 118, 151,  22, 108, 118, 153,  22, 108, 118, 155,  29,
         108, 118, 157,  29, 108, 118, 159,  29, 108, 118, 161,  25],
        [108, 118, 163,  25, 108, 118, 165,  25, 108, 118, 167,  25, 108, 118,
           4, 137,  20, 108, 118,  33, 108, 118, 139,  20, 108, 118, 141,  22,
         108, 118,  30, 108, 118, 145,  20, 108, 118, 147,  20, 108, 118, 149,
          22, 108, 118,  30, 108, 118, 153,  20, 108, 118, 15

In [7]:
generated_sequence = model.generate(gen_seed[:20], max_new_tokens=128)
print(generated_sequence[0].shape)
out = generated_sequence[0].cpu().numpy().tolist()
print(len(out))
gen_midi = tokenizer.tokens_to_midi(out)
gen_midi.dump('musicGPT.mid')

torch.Size([2, 252])
2


In [8]:
mixer.init()
mixer.music.load("musicGPT.mid")
mixer.music.play()

In [9]:
mixer.music.stop()

# Z manipulation

In [10]:
_ , small_data = create_subsets(dataset, [0.1])

In [11]:
z, labels = process_dataset_for_z(small_data)

In [12]:
get_artist_label.artist_id_mapping

{'Slayer': 0,
 'Judas Priest': 1,
 'Black_sabath': 2,
 'Pantera': 3,
 'Ozzy Osbourne': 4,
 'Sepultura': 5,
 'Children Of Bodom': 6,
 'Carcass': 7,
 'Megadeth': 8,
 'Type O Negative': 9,
 'midi_pop_songs': 10,
 'Mozart': 11,
 'Ravel': 12,
 'Dvorak': 13,
 'Beethoven': 14,
 'Haydn': 15,
 'Schubert': 16,
 'Cambini': 17,
 'Bach': 18,
 'Brahms': 19,
 'Faure': 20}

In [13]:
pointer_dict = calculate_feature_pointers(z,labels)

In [14]:
pointer_dict.keys()

dict_keys(['Beethoven', 'Children Of Bodom', 'Cambini', 'Schubert', 'Sepultura', 'Carcass', 'Slayer', 'midi_pop_songs', 'Brahms', 'Ravel', 'Judas Priest', 'Megadeth', 'Black_sabath', 'Ozzy Osbourne', 'Faure', 'Pantera', 'Dvorak', 'Bach', 'Haydn', 'Mozart', 'Type O Negative'])

In [17]:
pointer = pointer_dict['Dvorak']
magnitude = 1
pointer = pointer.tile(1)

In [18]:
model.generate(gen_seed ,max_new_tokens=128, latent_vector=pointer, magnitude= magnitude)

magnitude 1
z_vector tensor([ 8.0658, -4.8176,  2.3943, -3.2991,  6.9716,  1.0544,  7.8009, -8.0771,
         3.0934,  4.1215,  3.4642, -4.1732,  3.0921,  7.7531,  0.2165,  2.2996,
        -2.2709, 11.2617,  3.3499, -4.2135,  3.9536,  3.3372,  9.7705, -3.3861,
        -1.2279,  4.3181,  2.6494, -2.6907, -1.4028,  2.9462,  8.1385, -1.6483,
        -6.3669,  5.8235,  4.6581, -3.5691,  3.3750,  2.4097,  0.8522,  4.2377,
        -4.7078,  4.1051,  4.5070,  9.7888, -8.0963,  2.5936,  7.5204,  1.5311,
        -6.6679,  4.4963,  8.1422,  0.2520, -9.7220,  8.2531, -1.3385,  9.3462,
        -7.0001,  8.3142,  1.9845, 11.4923, -4.7780,  2.9750,  6.6355,  4.6584,
        -1.2829,  3.4960, -1.7008,  7.3620, -3.9086,  7.1093, -0.7029,  2.4259,
         7.5590, -3.9390, -2.0379,  9.5034, -0.2200, -7.1290,  3.1868,  2.9409,
        -3.9810,  1.1169,  2.1267,  1.8067,  6.5382, -6.2372,  3.8690,  2.6369,
         3.1495, -1.8014,  7.3941, -1.4862, 10.2907, -6.2188,  7.7997,  0.2584,
         4.3645, -8

RuntimeError: The size of tensor a (32) must match the size of tensor b (123) at non-singleton dimension 1